In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama3-8b-8192", groq_api_key=api_key)

In [ ]:
from langchain.schema import SystemMessage, AIMessage, HumanMessage
chat_message = [
    SystemMessage(content ="You are a helpful assistant that summarizes speeches."),
    HumanMessage(content= f"Please provide a short and concise summary of following speech: {speech}"),
]

In [12]:
llm.get_num_tokens(speech)

394

In [13]:
llm(chat_message).content

"Here is a concise summary of the speech:\n\nThe speaker, standing with pride and determination, emphasizes India's rich civilization and its potential to become a global leader in the 21st century. The country's vision is to provide equal opportunities for all citizens, with a focus on infrastructure development, women's empowerment, and sustainable growth through renewable energy and green technologies. The speaker highlights the power of Indian youth, urging citizens to dream big, believe in themselves, and work hard for the nation's progress. The ultimate goal is to build a nation where the aspirations of 1.4 billion people can thrive, and the speaker calls on the nation to unite and work together to achieve this vision."

In [14]:
# prompt template text summarization
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

generictemplate = """ Write a concise summary of the following speech:
speech: {speech}
Translate this following speech to {language} """

prompt = PromptTemplate(input_variables=["speech", "language"], template=generictemplate)
print(prompt)

input_variables=['language', 'speech'] input_types={} partial_variables={} template=' Write a concise summary of the following speech:\nspeech: {speech}\nTranslate this following speech to {language} '


In [19]:
prompt_format = prompt.format(speech = speech,language="Hindi")
llm_chain = LLMChain(llm=llm, prompt=prompt)
summary = llm_chain.run(speech=speech, language="Hindi")
print(summary)

Summary:

The speaker, filled with pride and determination, emphasizes India's rich history and civilization. They highlight the country's progress in the last decade, focusing on infrastructure development, women's empowerment, farmer support, and youth employment. They also emphasize sustainable growth, with a focus on renewable energy and environmental protection. The speaker believes that India's youth, with their creativity and innovation, hold the key to the country's success. They urge citizens to dream big, believe in themselves, and work hard for the nation's progress, ultimately aiming to build an India where all 1.4 billion people can thrive.

Hindi Translation:

मेरे प्यारे दोस्त,

आज मैं आपके सामने आ रहा हूँ, मेरे दिल में गर्व और निश्चय की भावना है. भारत नहीं sadece एक देश है, बल्कि सदियों से दुनिया को प्रेरित करने वाली एक संस्कृति है. हम संत और 學者, किसान और सैनिक, नवाचार और बलिदान के लिए प्रसिद्ध हैं. और अब हम संकल्प के साथ आगे बढ़ रहे हैं कि इस शताब्दी को भारत की शताब्दी

In [21]:
from langchain_community.document_loaders import PyPDFLoader
docs = PyPDFLoader("apjspeech.pdf").load_and_split()
print(docs)


[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [22]:
# pass the documents into prompt template
template = """ Write a concise summary of the following speech:
speech:{text}
"""

prompt = PromptTemplate(input_variables=["text"], template=template)

In [24]:
from langchain.chains.summarize import load_summarize_chain
# chain_type takes all the docs
chain = load_summarize_chain(llm = llm, chain_type = "stuff",verbose=True,prompt = prompt)
output_summary = chain.run(docs)
print(output_summary)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Write a concise summary of the following speech:
speech:A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions,

In [25]:
# Map reduce to summarize large documents
# convert the docs into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 100)
split_docs = text_splitter.split_documents(docs)
print(split_docs)

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [26]:
# this will be my final prompt that summarizes all the chunks and give the final summary
final_prompt = """ Provide the final summary of the entire speech with these important points:
Add a motivation title, Start the precise summary with an introduction and provide the summary in number
points for the speech
Speech:{text}"""

final_prompt_template = PromptTemplate(input_variables=["text"], template=final_prompt)

In [28]:
chunks_prompt_template = """ Write a concise summary of the following speech:
speech:{text}
Summary: """

chunk_prompt = PromptTemplate(input_variables=["text"],template=chunks_prompt_template)
map_reduce_chain = load_summarize_chain(llm = llm, chain_type = "map_reduce", verbose = True, map_prompt = chunk_prompt,combine_prompt = final_prompt_template)
output = map_reduce_chain.run(split_docs)
print(output)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Write a concise summary of the following speech:
speech:A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interacti

Token indices sequence length is longer than the specified maximum sequence length for this model (1465 > 1024). Running this sequence through the model will result in indexing errors




> Entering new LLMChain chain...
Prompt after formatting:
 Provide the final summary of the entire speech with these important points:
Add a motivation title, Start the precise summary with an introduction and provide the summary in number
points for the speech
Speech:Here is a concise summary of the speech:

President A.P.J. Abdul Kalam reflects on his five-year tenure at Rashtrapati Bhavan, expressing gratitude to the diverse individuals he has interacted with. He highlights 10 key messages he has learned during his interactions, including the importance of accelerating development, empowering villages, and mobilizing rural core competence for competitiveness. He also shares a personal anecdote about a young girl's question about India's potential to become a developed nation by 2020, which he assures her is a goal worth striving for.

Here is a concise summary of the speech:

The speaker appreciated the question and assured the person that their dream would be taken to the highest

In [29]:
# for refine chain summarization
refine_chain = load_summarize_chain(llm = llm,chain_type="refine",verbose = True)
output_summ = refine_chain.run(split_docs)
print(output_summ)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have man